# Redes bayesianas - Resident Evil

In [39]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta

In [40]:
import pgmpy.readwrite.XMLBIF as rwBIF

reader = rwBIF.XMLBIFReader('residentevil.xml')
Modelo_residentevil = reader.get_model()

In [41]:
Modelo_residentevil.cpds[2]

<TabularCPD representing P(Horda:2 | Ciudad:2, Lugar:3) at 0x11b956ac8>

In [64]:
Modelo_residentevil.check_model()



/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


True

In [43]:
#Función que descarta las variables irrelevantes

def descarta(m, c, e):
    lista = c+e
    descartados = []
    padres = []
    for l in lista:
        subgrafo = Modelo_residentevil.subgraph(networkx.ancestors(Modelo_residentevil, l))
        for s in subgrafo:
            padres.append(s)
                          
    for n in m.nodes():
        if n not in lista and n not in padres:
            descartados.append(n)
                  
    return descartados  

consulta = []
consulta.append("Refugio")
evidencias = []
evidencias.append("Cooperativo")
evidencias.append("Medicina")
descarta(Modelo_residentevil,consulta,evidencias)

['Horda', 'Agua', 'Comida', 'Vehiculo', 'Armas', 'Soldados umbrella']

In [44]:
#Función que devuelve los factores iniciales

def factores_iniciales(m):
    d = descarta(m,consulta,evidencias)
    c = 0
    factores = []
    for n in m.nodes():
        if n not in d:
            factores.append(m.cpds[c].to_factor())
        c+=1
    return factores


In [61]:
#Eliminación de variables: Uso de la clase VariableElimination

Modelo_re_ev = pgmi.VariableElimination(Modelo_residentevil)
consulta = Modelo_re_ev.query(['Refugio'], {'Ciudad': 0, 'Cooperativo': 0})
print(consulta['Refugio'])

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.8889 │
├───────────┼────────────────┤
│ Refugio_1 │         0.1111 │
╘═══════════╧════════════════╛


/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
